# 为什么我还是无法理解transformer？

 **Author:** [vtch]

 **Link:** [https://www.zhihu.com/question/596771388/answer/119375053579]


> 偶然间在网上看到了个用excel手搓Transformer的Yeh大神，他居然把Transformer给excel化了，只是非常遗憾没有任何注释，地址请见后面的参考文献。

![](img_transformer/v2-f60a4438d787adb024187feaee857c85_720w.jpg)  


致敬Prof.Tom Yeh

  
  
结合Transformer论文和Tom Yeh大神的开源文档，配合矩阵计算器，我彻底把它弄懂了，整理好贴在下面，有兴趣的观众可以自行验算，相信对于理解Transformer非常有帮助。

![](img_transformer/v2-82263a1acdc93c721799eccab132b913_720w.jpg)  


Transformer架构图 

  
  

> 参考：[https://zhuanlan.zhihu.com/p/17862781045](https://zhuanlan.zhihu.com/p/17862781045)

## 1 背景信息  
### 1.1 输入输出  
![](img_transformer/v2-c43355de361bcd4292e813fc35375e3c_720w.jpg)  


背景: 输入输出的案例

  
  
### 1.2 Vocabulary字典  
字典就就是你输入的词汇的对应关系，也就是Tokenize和Embedding的故事，这个后面会详细讲，这里简化了，就是个示意图，实际上词汇表都很大，而且对应想embedding向量非常大。

![](img_transformer/v2-1aca4d7c286176839707b93fbc4e0a36_720w.jpg)  


字典

  
  

> **关于字典：**tokenize和embedding可以选择不同的字典，这里选择了同一个，实际上很多大模型也选择了同一个，因为方便。

### 1.3 流程说明  
Transformer可以看到是分左右两列的，左边的叫Encoder，右边的叫Decoder，数据流分别都是从下到上，本身是可以多层堆叠的，在实务中可以选择只实现Encoder，也可以只实现Decoder。

Encoder部分的输入是一次性输入的，模型瞬间就知道了所有的输入，完成了计算**它的最后的输出也会分别进入Decoder每一层**（在Decoder中详解）。

Decoder部分和Encoder类似，但是因为它是自回归模型，所以它的输入就是它是输出，由于时序的问题，它输入是比输出晚一个token的，所以第一个Decoder的输入就是个空Token（叫[S]）。

Decoder的输出其实就是基于Encoder的输入，和它每一个token的输出来共同决定的，也就是要一个token一个token的输出。

## 2Encoder和Decoder的编码处理  
### 2.1 整体说明  
这部分讲的是讲输出输出数据进行预处理，如下图。

![](img_transformer/v2-1058795a61dd64533dafa8291430070c_720w.jpg)  


Tokenization, Embeding and Positional Encoding

  
  
这部分的输入其实有两部分，一部分是标准的输入，另外由于Transformer的decoder部分是自回归，所以输出会逐个的作为decoder的输入返回给Transformer，它是不需要考虑Tokenization的（严谨吧？）。

在正式进入大模型输入前，还要做三件事：

1. **Tokenize化：**将输入的raw text标准化。
2. **Embedding：**将自然语言向量化，可以得到一个矩阵，将输入的token用另一个维度来表示
3. **Positional Encoding：**导入位置信息。

### 2.2 tokenization  

> 请参考我的回答：[https://www.zhihu.com/question/64984731/answer/114672307054](https://www.zhihu.com/question/64984731/answer/114672307054)

这里的输入和输出比较简单，可以直接按照单词、字分解为字典里的token，这部分就是按照字典展开，取决模型自己的设计，没有固定模式。

### 2.3 Embedding  

> 请参考我的回答：[https://www.zhihu.com/question/38002635/answer/114206640524](https://www.zhihu.com/question/38002635/answer/114206640524)

请观察下面字典的例子:

例如**“狗”**可以用一个**Embedding向量[3,0,0,0,1,3]**来表示（当然实务中可能是原论文的 $512$ 维甚至更高）

![](img_transformer/v2-734cdd88756084881508b2ba2393ac0f_720w.jpg)  


Vocabulary的例子

  
  

> 字典的词数叫**Vocabulary size**，其中单语言模型 30k-50k，多语言模型 100k-250k。

有必要多说一下这个维度：

1. **embedding的维度：**表示用多少维的向量表示一个token。
2. **模型内部的标准维度d\_model：通常和embedding一致。**
3. **如果不一致也没关系，可以通过矩阵把它映射成同一维度（如本文就是这样的情况）**

Encoder的embedding结果是一个矩阵，见于红色框线内，后面还会做Positional Encoding。

![](img_transformer/v2-5497625af5e7d96689dca847a8af6247_720w.jpg)  


Encoder部分的Tokenization & Embedding

  
  
Decoder的自回归输入到decoder的embedding结果是一个矩阵，见于红色框线内，后面还会做Positional Encoding。

![](img_transformer/v2-90d5820057b7b83f1761cd4556adb767_720w.jpg)  


Output Embedding

  
  
### 2.4PositionalEncoding  

> 请参考我的回答：[https://www.zhihu.com/answer/115575929186](https://www.zhihu.com/answer/115575929186)

因为注意力机制是不考虑Token位置的，这个就很可怕了，主语宾语都不分的话，那肯定无法理解意思的，所以呢，一句话的位置信息也是很重要的，到底怎么考虑，其实都可以的，只要能分得清楚即可（就是不一样就行，不要管实际的大小数值）。

PE 的值可以通过训练得到，也可以使用某种公式计算得到。在 Transformer 中采用了后者，位置向量有多种表示方式。其中一种常见的是正弦 - 余弦位置编码（Sinusoidal Position Encoding）。 位置编码的公式如下（这个公式看着挺好的，比起单纯的数值，都不需要考虑位数的问题，容量无限扩展）： 

$$PE_{pos,2i}=\sin(pos/10000^{2i/d_{model}})\newline PE_{pos,2i + 1}=\cos(pos/10000^{2i/d_{model}})$$ 

我这里的 $d_{model}=6$ ，一共 $6$ 个tokens：

![](img_transformer/v2-5310446bdc4d1ab97bcd7c1a39a2dcab_720w.jpg)  


正弦 - 余弦位置编码案例

  
  
这里的计算显然没有考虑正弦-余弦公式（我只能假设是Yeh老师自己设计的，其实不影响结果的）。

![](img_transformer/v2-9277206dd288ab6070a24c6afb5dec35_720w.jpg)  


Encoder部分Positional Encoding

  
  
![](img_transformer/v2-e7ac62b428d469df5b5475611c914920_720w.jpg)  


Decoder部分Positional Encoding

  
  
## 3 Encoder  
### 3.1 Encoder整体说明  
![](img_transformer/v2-25f15c842f56489dcb56c32057bc4979_720w.jpg)  


Transformer Encoder部分说明

  
  
Transformer的Encoder总共分两个阶段，先做多头注意力，再做FFN，这两步结束后都要做Add & Norm，主要的功能就是做归一化和标准化，提升模型的稳定性和训练速度。

需要专注的是多头注意力机制（Multi-Head Attention, MHA），它其实是多个注意力头拼接（concat）的，对于每个注意力头的机制都有详细的过程。


> Encoder部分如果理解清楚了，Decoder就非常容易了。

### 3.2Mult-HeadAttention  
![](img_transformer/v2-e779870ff2cdce2894e4845859218cc4_720w.jpg)  


QKV机制

  
  
这里可以理解为将每个输入向量（维度为embedding）映射到三个键向量（维度为d\_model/head）。

* **Query（查询）**：代表当前关注的位置（如当前处理的单词）。
* **Key（键）**：代表序列中所有位置的标识，用于与 Query 匹配相似性。
* **Value（值）**：存储实际的信息，通过注意力权重加权后生成最终输出。

![](img_transformer/v2-2b2c6bb9013fedda70ee852054f12874_720w.jpg)  


Q的计算

  
  

> Yeh大神的做法是都做了转置的，我已经用公式做了说明，后面每个计算都是类似的方法，就没有一一赘述了（以防我可能有的地方忘了，我尽量说清楚）。

后面就是所谓的d\_model了，这里是 $d_{model}=6$ ，不信看下下面的W矩阵结构就清楚了，有 $2$ 个注意力头，每个注意力头是 $3$ 维，等于将embedding的维度用 $2\times3=6$ 维来代替了（如果你没有理解，后面走完流程相信你也能理解。）

![](img_transformer/v2-d1a0b662d3cc5350b8213207d13768f3_720w.jpg)  
同理得到K、V。

![](img_transformer/v2-55c5498b199b21182d49b8da2f82f5b3_720w.jpg)  


K、V结果

  
  
![](img_transformer/v2-f4bdfad317e3a6bdff8fa030fcac266b_720w.jpg)  


scaled Dot-Product Attention说明

  
  

> 仔细看下Q、K、V是怎么计算的，其实也是做了转置的，见下面的公式。

![](img_transformer/v2-6305e05ef52b7c5ffefa3c5f9e51f237_720w.jpg)  


Scaled Dot-Production Attention公式

  
  
下面从 Head 0 开始，看看是怎么计算 Scaled Dot-Production Attention的。

![](img_transformer/v2-b119956da53f762ca243893246ed6b85_720w.jpg)  


Scaled Dot-Production Attention公式

  
  
同理可以计算 Head 1，得到结果拼接后做多头注意力。

![](img_transformer/v2-72ac04b0b55f9cd5f1b9ed1b2aeaae43_720w.jpg)  


多头注意力

  
  
也需要转置。

![](img_transformer/v2-40ef56c91da1063c04b42f1cd32c8156_720w.jpg)  


多头注意力公式

  
  
接着做线性变换，将拼接的矩阵维度，从 $h\times d_v = 2\times3 = 6$ ，重新投影到 $5$ 维矩阵，这样的结果就和输入很像了，每列代表一个token的注意力结果。


> 你可能有这个问题：为啥要做维度变换呢？（这里我只解释一次，后面会有很多次相同的操作。）  
> 答案：因为它需要做add，即把最初的输入加进去，这样下一层就相当于得到了本层的结果和最初的输入信息，免得丢失信息，既然要做加法，自然结果要变成和输入一致。

![](img_transformer/v2-7f21b4c05a912795bc9a99f43bc781cc_720w.jpg)  


线性变换

  
  
### 3.3 Add&Norm  
这个Add & Norm层其实是神经网络网络中的一个基操，目的只是为了训练的稳定来做的，在实务中，很多大型神经网络的Norm是在Attention之前做的，例如LLaMa，它是先做Norm再做Add的。

![](img_transformer/v2-e31c5853ce203c5326dc40da5e299e16_720w.jpg)  


LLaMa架构 | 来源：网络

  
  
Add 叫做残差连接是一种在深度神经网络中常用的技术，将输入直接传递到输出，与经过网络层变换后的结果相加，目的就是让多层的神经网络也能感受到最初的输入。


> 残差连接（Residual Connection）最早由何恺明等人在2015年提出的ResNet（残差网络）中引入。它的核心思想是通过引入“跳跃连接”（Skip Connection），从而缓解深度网络中的梯度消失或爆炸问题。

![](img_transformer/v2-aa07e586b9bbbb3784f958417ba0a972_720w.jpg)  


Add & Norm

  
  
这里其实还有个操作在Add & Norm层中，叫做Scaled & shift（缩放和偏移），其核心作用是对输入数据进行标准化处理，以提高模型的稳定性和训练效率。


> 就是缩放+偏移值，效果就是提升模型的稳定度，这个是深度学习的技巧了，可以忽略。

  


![](img_transformer/v2-8dc0ba72b86bc912d6afd82e58a9223a_720w.jpg)  


Scaled & shift（缩放和偏移）

  
  
### 3.4 Feed Forward Neutral Network  
以上注意力就完成了，我们走到下面的FFN（涉及到深度学习的部分我们先不展开，只要知道它是全互联就好了。）

1. **第一层全连接**：将输入升维到 $4\times d_{model}$ ，这里放大 $2$ 倍。
2. **ReLU**：引入非线性，过滤负值。
3. **第二层全连接**：降维回 $d_{model}$ 。

![](img_transformer/v2-e22297fe734e0b9296a59ca09c46585a_720w.jpg)  


FFN处理

  
  
全连接要干的事就是不要丢掉信息，所以要放大维度，然后在整理会标准的维度处理，实际是个数学问题，我简单给点背景信息，不用纠结，知道大概要怎么算就好，变换里带了两个偏移量，都是为了提升模型稳定度的。


> $b_1$ 提供了额外的自由度，允许模型对输入进行平移修正，增强对数据分布的适应性，虽然理论上可以赋0，但在实践中必须保留它。偏置项是神经网络中实现灵活特征变换的核心组件，移除它会损害模型的表达能力。这种设计是深度学习模型在大量实验验证后的最佳实践。  
>  $b_2$ 的作用虽不如b1关键，但它提供了输出层的平移自由度，帮助模型适配目标分布偏移。实践中应保留除非在极端资源受限的场景中且实验证明移除后性能损失可接受。

### 3.5 Add&Norm  
还是Add & Norm，再来一次Scaled & Shift。得到的结果见红框，它是给Decoder作为输出的，它和输入的维度是一致的，就像刚刚embedding完。

![](img_transformer/v2-ddf2884485d9f9e59e6451c80eb9975e_720w.jpg)  


Encoder最后输出

  
  
## 4 Decoder  
### 4.1 Decoder 说明  
![](img_transformer/v2-c9ca0b454bd3b2fd396e6ac9d8753843_720w.jpg)  


Decoder框架

  
  
Decoder部分和Encoder非常相似，可以看到除了前面的Masked MHA以外，和Encoder一摸一样的，所以 **Decoder = Masked MHA + Encoder。**

Mashed MHA输出只负责下一层“Encoder"的Q，其中的 K、V值来自Encoder的最后输出，所以对于我们讲解过的Encoder部分一带而过。

### 4.2 MaskedMulti-HeadAttention  
其实Masked MHA的核心还是scaled dot-production attention，首先还是制备Q、K和V。

![](img_transformer/v2-883fb2f7c62800be071e25707492023a_720w.jpg)  


计算Q、K、V

  
  
得到了Q、K、V之后，进行Masked MHA计算，实际很简单，只是做个矩阵加法，增加一个极大负值，注意转置的问题。

![](img_transformer/v2-64355c3f4121558e89fff68981152327_720w.jpg)  


Scaled dot-production attention打开Mask

  
  
看下实际的计算，依然是以Head 0 为例，注意Casual Mask也做了转置。需要注意的是这里使用了 $3$ 个注意力头，后面还会切换成 $2$ 个注意力头，非常灵活。

![](img_transformer/v2-faa223c5ef123b684fd5b4582b49fde0_720w.jpg)  


计算Masked 

  
  
同理得到 Head 1、Head 2 的结果，通过线性变换搬回到系统维度 d\_model。

![](img_transformer/v2-59a0fbabd6d3238abc4cc1e06aca315c_720w.jpg)  


从3个注意力头切换到系统维度 d\_model = 6

  
  
Add。

![](img_transformer/v2-d8e221c1bfee158a9ea17cc21ab5d6cc_720w.jpg)  


add

  
  
Normal，Scale & shift。

![](img_transformer/v2-17ff86eba3cd8a0e5e5b65fac6a53e70_720w.jpg)  


Normal，Scale & shift

  
  
### 4.3 Multi-Head Cross-Attention  
之所以叫cross，就是因为其中的K、V来自Encoder部分的最后输出，Q来自Decoder的输入，实际上，它和Encoder中的MHA其实计算上没有区别。

![](img_transformer/v2-a64348ad61e02bd30e2a1c0e3829d472_720w.jpg)  


Multi-Head Cross-Attention

  
  
计算标准的MHA，以 head 0 为例。

![](img_transformer/v2-639a55331a993346bbbd5017ca31bfe1_720w.jpg)  


以Head 0为例计算MHA

  
  
同理计算 head 1，并做线性变换到系统维度。

![](img_transformer/v2-e6c40ebf39cf686a12a9cf034c35f439_720w.jpg)  


线性变换

  
  
Add。

![](img_transformer/v2-72b9cc723c39a5b557dda62d009fd14f_720w.jpg)  


add

  
  
Normal，scale & shift 。

![](img_transformer/v2-b8b7cc6b0380cae5c978551ebb3f80bb_720w.jpg)  


Normal，scale & shift

  
  
### 4.4 FeedForwardNeutral Network  
依然是扩维到 $2\times d_{model} = 12$ ，维度不够，拿 1 来凑。

![](img_transformer/v2-d515ca609c03f14b31ab2ab9b61b4a05_720w.jpg)  


扩维

  
  
再变换回去。

![](img_transformer/v2-a56c45931e897ecc92b037d973863e79_720w.jpg)  


调整到系统维度

  
  
Add。

![](img_transformer/v2-5befd25cb9fa859340a04b12b93ab982_720w.jpg)  


add

  
  
Normal，scale & shift。

![](img_transformer/v2-a235b361a444c1127666372357909ea3_720w.jpg)  


Normal，scale & shift

  
  
## 5 Output处理  
### 5.1Linear  
这里的线性变换无非是为了凑到输出的 字典，因为现在要输出的只是一个token。

![](img_transformer/v2-249d5e65c97523d6f40660292e92c15d_720w.jpg)  


线性变化

  
  
### 5.2Softmax  

> 请参考我的回答：[https://www.zhihu.com/question/23765351/answer/113490283504](https://www.zhihu.com/question/23765351/answer/113490283504)

![](img_transformer/v2-046436de868ed6bafc5d18f127a30f72_720w.jpg)  


得到结果

  
  
这个结果其实有问题的，因为是输出是一个token一个token进去的，Decoder部分算上PE，总共成了 $7$ 个token，其实应该是5个，不过问题不大，并不影响理解。

实务中就是最后一列可能性最大的token就是输出的token。

~结束~

## 6 其他几个问题  

> 这个问题都挺大，篇幅有限，后面慢慢找时间再细讲。

### 6.1 模型到底多大  
模型参数指的就是那些 $W$ 矩阵，这里都是转置形式体现的，这些 $W$ 矩阵的每一个值就是一个parameter，加在一起就是模型参数量。

从整体上看，系统维度是 d\_model，但是中间做了太多次变换了，感兴趣的各位可以自己去计算。

实务中可能每一层并不一样，比如DeepSeek有61层的Decoder，并没有Encoder，但是这61层前四层和后57层又不一样，非常复杂。

### 6.2 模型的权重是怎么更新的  
这个就要涉及到模型训练了，有三个问题：

* **前向传播（Forward Propagation）：**

输入到输出各层计算（如卷积、池化层等），产生输出并完成损失函数计算。

* **反向传播（Back Propagation）：**

由输出到输入反向完成整个模型中各层的权重和输出对损失函数的梯度求解。

* **梯度更新（Weight Update）：**

根据指定的指定学习率，对模型权重通过梯度下降算法完成权重的更新。

以上的都是有框架比如PyTorch自动给你算出来，刚开始每个权重矩阵都是随机值，后面每运行一次，都可以改变一次矩阵值，该怎么改就要看你的梯度了，按道理来讲要让梯度变成 $0$ ，但是变得太快也不行，会走得过头了，怎么改就要看你的优化器算法了，本质是数学问题，作为硬件工程师，我其实并不关心，我关注的是它对于我们硬件需求有什么变化。

请参考下面：


> [https://zhuanlan.zhihu.com/p/15842434908](https://zhuanlan.zhihu.com/p/15842434908)

### 6.3 KV cache到底是什么  

> 请参考计算KV显存需求：[https://www.zhihu.com/question/12032876690/answer/102742235871](https://www.zhihu.com/question/12032876690/answer/102742235871)

研究清楚了整个推理过程之后，就知道KV cache是什么了。

![](img_transformer/v2-8a364e096a981be8f96567789fd4c341_720w.jpg)  


从Transformer架构看KV Cache

  
  
对于一次推理过程，Decoder每输出一个token就要做一次运算，你会发现这个K和V都来自Encoder部分，其实就是输入序列算出来的，每一层的Decoder都有一份，那就没有必要每次都重算一遍了，直接把这批KV存起来就可以了，那每一层有多少个kv cache呢？你可以看到其实就是d\_model个，也就是所谓的hidden\_size，理解KV cache的显存计算就再也没有问题了。

## 7 参考  
1. [https://hellogithub.com/repository/65ff1166e0c3457e9d956bdecbd84a35](https://hellogithub.com/repository/65ff1166e0c3457e9d956bdecbd84a35)
2. [https://arxiv.org/pdf/1706.03762.pdf](https://arxiv.org/pdf/1706.03762.pdf)
3. [https://aclanthology.org/D13-1176.pdf](https://aclanthology.org/D13-1176.pdf)
4. [https://arxiv.org/pdf/1406.1078.pdf](https://arxiv.org/pdf/1406.1078.pdf)
5. [https://arxiv.org/pdf/1409.3215.pdf](https://arxiv.org/pdf/1409.3215.pdf)
6. [https://arxiv.org/pdf/1409.0473.pdf](https://arxiv.org/pdf/1409.0473.pdf)
7. [https://chenzomi12.github.io/](https://chenzomi12.github.io/)
